In [ ]:
!pip install torch transformers pandas
!pip install matplotlib

In [ ]:
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from transformers import BertTokenizerFast,AutoModel,BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
import requests
import zipfile
import re
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import itertools
from torch.nn import CrossEntropyLoss


In [ ]:
request = requests.get("https://github.com/MHDBST/PerSenT/archive/refs/heads/main.zip")
with open("data.zip", "wb") as file:
    file.write(request.content)

# Unzip data
with zipfile.ZipFile('data.zip') as zip:
    zip.extractall('data')

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# This class is responsible for reading the CSV file containing paragraph data,
# tokenizing paragraphs using the provided tokenizer, and returning the tokenized input along with the sentiment label.
class PerSentParagraphDataset(Dataset):
    def __init__(self, path, tokenizer, max_length=150): #256
        # seperate data by columns.
        self.data = pd.read_csv(path, delimiter=',')
        self.data.dropna(subset=['DOCUMENT','TRUE_SENTIMENT'], inplace=False)
        self.tokenizer = tokenizer
        self.max_length = max_length

        # Collect paragraphs and their sentiments
        self.paragraphs = []
        self.sentiments = []
        
        for _, row in self.data.iterrows():
            #devide each paragraph from document.
            paragraphs = row['DOCUMENT'].split('\n')
            for i in range(len(paragraphs)):
                if(i >=16):
                  break
                paragraph_col = f'Paragraph{i}'
                if pd.notna(row[paragraph_col]):
                    #store its paragraph
                    self.paragraphs.append(paragraphs[i])
                    #get each sentiments 
                    self.sentiments.append(row[paragraph_col])
                # if there is one paragraph, then that paragraph is the document sentiment.
                elif (len(paragraphs) == 1): 
                    self.sentiments.append(row['TRUE_SENTIMENT'])  
                    self.paragraphs.append(paragraphs[i])
                    
    def __len__(self):
        return len(self.paragraphs)

    def __getitem__(self, idx):
        paragraph = self.paragraphs[idx]
        true_sentiment = self.sentiments[idx]
        # make paragraph as caculatable form
        inputs = self.tokenizer.encode_plus(
            paragraph,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors='pt'
        )
        # laveling
        label_map = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
        label = torch.tensor(label_map[true_sentiment], dtype=torch.long)
        
        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'labels': label
        }


In [ ]:

def plot_confusion_matrix(cm, target_names):
    plt.imshow(cm, interpolation='nearest', cmap=plt.get_cmap('Blues'))
    plt.colorbar()
    tick_marks = np.arange(len(target_names))
    plt.xticks(tick_marks, target_names, rotation=45)
    plt.yticks(tick_marks, target_names)

    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > cm.max() / 2 else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()

In [ ]:
# Training and validation function for combined datasets
def fine_tune(model, train_loader, val_loader, num_epochs=4, device=None):
    if device is None:
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = model.to(device)
    optimizer = AdamW(model.parameters(), lr=2e-5)
    weighted_loss = CrossEntropyLoss(weight=torch.tensor([3.0, 1.0, 1.0]).to(device))

    for epoch in range(num_epochs):
        # Training loop
        model.train()
        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        # Validation loop
        true_labels = []
        pred_labels = []
        model.eval()
        total_correct = 0
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            with torch.no_grad():
                outputs = model(input_ids, attention_mask=attention_mask)
                _, predictions = torch.max(outputs.logits, dim=1) # argMax
                total_correct += torch.sum(predictions == labels).item()
            true_labels.extend(labels.cpu().numpy())
            pred_labels.extend(predictions.cpu().numpy())

        accuracy = total_correct / len(val_loader.dataset)
        report = classification_report(true_labels, pred_labels, target_names=['Negative', 'Neutral', 'Positive'])
        cm = confusion_matrix(true_labels, pred_labels)
        
        print(f'Epoch {epoch + 1}/{num_epochs}')
        print('Report: ');
        print(report)
        print('Confusion Matrix:')
        plot_confusion_matrix(cm, target_names=['Negative', 'Neutral', 'Positive'])


    return model

In [ ]:

# # Load BERT and tokenizer: The pre-trained BERT model and tokenizer are loaded
# # using the 'BertTokenizerFast' classes from the Hugging Face Transformer library
# tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

train_dataset = PerSentParagraphDataset('data/PerSenT-main/train.csv', tokenizer)
val_dataset = PerSentParagraphDataset('data/PerSenT-main/dev.csv', tokenizer)
# # Create data loaders: Train and validation datasets are created using the custom dataset class.
# # Data loaders are created with a specified batch size and shuffling for the traing set.
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=False)



In [ ]:

#initilize model and optimizer: The Bert model for sequence classification is initialized with three output lavels.
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
fine_tuned_model = fine_tune(model, train_loader, val_loader)

In [ ]:
# Save the fine-tuned model
fine_tuned_model.save_pretrained('fine_tuned_bert_sentiment_elm')

# model.save_pretrained('fine_tuned_bert_sentiment')

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('fine_tuned_bert_sentiment_elm')

def predict_paragraph_sentiments(document):
    paragraph_sentiments = []
    paragraphs = document.split('\n')
    true_labels = []
    pred_labels = []
    for paragraph in paragraphs:
        if len(paragraph.strip()) > 0:
            inputs = tokenizer.encode_plus(
                paragraph,
                padding='max_length',
                truncation=True,
                max_length=130,
                return_tensors='pt'
            )

            with torch.no_grad():
                outputs = model(**inputs)
                _, predictions = torch.max(outputs.logits, dim=1)
                sentiment = ['Negative', 'Neutral', 'Positive'][predictions.item()]
                paragraph_sentiments.append(sentiment)

    return paragraph_sentiments

def predict_document_sentiment(document):
    paragraph_sentiments = predict_paragraph_sentiments(document)
    # Positive -> Negative -> Neutral (if they have same number)
    sentiment_count = {'Positive': 0,'Negative': 0, 'Neutral': 0}
    
    for sentiment in paragraph_sentiments:
        sentiment_count[sentiment] += 1
    return max(sentiment_count, key=sentiment_count.get)
    

In [ ]:

def evaluate_document(path):
    data = pd.read_csv(path, delimiter=',')
    true_labels = []
    pred_labels = []

    for _, row in data.iterrows():
        true_labels.append(row['TRUE_SENTIMENT'])
        pred_labels.append(predict_document_sentiment(row['DOCUMENT']))

    label_map = {'Negative': 0, 'Neutral': 1, 'Positive': 2}
    true_labels = [label_map[label] for label in true_labels]
    pred_labels = [label_map[label] for label in pred_labels]

    accuracy = accuracy_score(true_labels, pred_labels)
    recall = recall_score(true_labels, pred_labels, average=None)
    f1 = f1_score(true_labels, pred_labels, average=None)
    f1_macro = f1_score(true_labels, pred_labels, average='macro')
    recall_macro = recall_score(true_labels, pred_labels, average='macro')
    
    return accuracy, recall, f1, recall_macro, f1_macro


In [ ]:
accuracy, recall, f1, recall_macro, f1_macro = evaluate_document('data/PerSenT-main/fixed_test.csv')

data = {
    'Metric': ['Accuracy', 'Recall (Negative)', 'Recall (Neutral)', 'Recall (Positive)', 'Macro Recall', 'F1 (Negative)', 'F1 (Neutral)', 'F1 (Positive)', 'Macro F1'],
    'Value': [accuracy, recall[0], recall[1], recall[2], recall_macro, f1[0], f1[1], f1[2], f1_macro]
}

results_df = pd.DataFrame(data)
print(results_df)
# print(f'Test accuracy: {accuracy:.4f} recall: {recall:.4f} f1-score: {f1:.4f}')